## Подготовка окружения

В данном ноутбуке реализуется модуль обработки аудиосообщения пользователя.
Для работы используются предобученные модели распознавания эмоций по речи и
преобразования речи в текст.

In [4]:
!pip install -q transformers librosa soundfile openai-whisper

In [5]:
import librosa
import numpy as np
import soundfile as sf

## Загрузка и ресемплинг аудиофайла

На текущем этапе аудиосообщение пользователя передаётся в виде файла.
Аудиосигнал приводится к частоте дискретизации 16 кГц для корректной
работы используемых моделей.

In [6]:
AUDIO_PATH = "/content/silents.wav"

audio, sr = librosa.load(AUDIO_PATH, sr=16000)

print("Sample rate:", sr)
print("Duration (sec):", len(audio) / sr)

Sample rate: 16000
Duration (sec): 2.6640625


## Валидация аудиосообщения

Перед дальнейшей обработкой аудиосообщение проверяется на корректность.
Отбрасываются записи, содержащие только тишину, слишком короткие записи
и записи с чрезмерным уровнем шума.

In [7]:
def is_silent(audio, threshold=0.003, min_ratio=0.05):
    rms = librosa.feature.rms(y=audio)[0]
    speech_ratio = (rms > threshold).mean()
    return speech_ratio < min_ratio

def is_too_short(audio, sr, min_duration=1.0):
    return len(audio) / sr < min_duration

def is_noisy(audio, threshold=0.4):
    zcr = librosa.feature.zero_crossing_rate(audio)
    return zcr.mean() > threshold

def validate_audio(audio, sr):
    if is_silent(audio):
        return False, "silence"
    if is_too_short(audio, sr):
        return False, "too short"
    if is_noisy(audio):
        return False, "too noisy"
    return True, "ok"

In [8]:
is_valid, reason = validate_audio(audio, sr)
print("Audio valid:", is_valid, "| Reason:", reason)

Audio valid: False | Reason: silence


In [9]:
AUDIO_PATH = "/content/my_audio_happy.wav"

audio, sr = librosa.load(AUDIO_PATH, sr=16000)

print("Sample rate:", sr)
print("Duration (sec):", len(audio) / sr)

Sample rate: 16000
Duration (sec): 3.0240625


In [10]:
is_valid, reason = validate_audio(audio, sr)
print("Audio valid:", is_valid, "| Reason:", reason)

Audio valid: True | Reason: ok


In [11]:
rms = librosa.feature.rms(y=audio)
print("RMS mean:", rms.mean())
print("RMS min:", rms.min())
print("RMS max:", rms.max())

RMS mean: 0.037251644
RMS min: 0.0
RMS max: 0.152562


## Распознавание эмоции по аудиосообщению

Эмоциональная окраска речи пользователя определяется на основе аудиосигнала
с использованием предобученной модели распознавания эмоций по речи.

In [12]:
from transformers import pipeline

emotion_model = pipeline(
    "audio-classification",
    model="superb/hubert-large-superb-er"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Device set to use cuda:0


In [13]:
emotion_result = emotion_model(
    {"array": audio, "sampling_rate": sr}
)

emotion_audio = emotion_result[0]["label"]
emotion_score = emotion_result[0]["score"]

print("Emotion:", emotion_audio)
print("Confidence:", round(emotion_score, 3))

Emotion: hap
Confidence: 0.762


## Нормализация аудиосигнала

Для повышения устойчивости распознавания речи выполняется нормализация
громкости аудиосигнала перед этапом Speech-to-Text.


In [14]:
def normalize_audio(audio, target_rms=0.1):
    rms = np.sqrt(np.mean(audio**2))
    if rms == 0:
        return audio
    return audio * (target_rms / rms)

audio_norm = normalize_audio(audio)
sf.write("normalized.wav", audio_norm, sr)

## Преобразование речи в текст (Speech-to-Text)

Аудиосообщение пользователя преобразуется в текстовую форму для дальнейшего
семантического анализа.


In [15]:
import whisper

whisper_model = whisper.load_model("small")

100%|████████████████████████████████████████| 461M/461M [00:04<00:00, 117MiB/s]


In [16]:
result = whisper_model.transcribe(
    "normalized.wav",
    language="ru",
    task="transcribe",
    condition_on_previous_text=False,
    temperature=0.0,
    no_speech_threshold=0.3
)

transcript = result["text"].strip().lower()

print("Transcript:")
print(transcript)

Transcript:
привет, дела!


## Валидация результата распознавания речи

Текстовая транскрипция считается корректной, если содержит осмысленную
речь достаточной длины.

In [17]:
def is_valid_transcript(text, min_words=2):
    return len(text.split()) >= min_words

print("Transcript valid:", is_valid_transcript(transcript))

Transcript valid: True


## Итоговый результат обработки аудиосообщения

In [18]:
output = {
    "emotion_audio": emotion_audio,
    "transcript": transcript
}

output

{'emotion_audio': 'hap', 'transcript': 'привет, дела!'}

Перед извлечением пользовательских предпочтений был проведён анализ музыкального датасета с целью определения допустимых категорий жанров и языков. Это позволило ограничить пространство возможных намерений пользователя только теми категориями, которые реально представлены в данных, и повысить согласованность между пользовательским запросом и этапом подбора музыкального контента.

Для извлечения жанровых предпочтений был сформирован словарь жанров на основе анализа распределения жанров в итоговом музыкальном датасете. Сырые жанровые теги были агрегированы в канонические жанровые классы (pop, rock, hip-hop, electronic и др.), отражающие реальные пользовательские формулировки. Извлечение жанра из пользовательского текста выполняется с помощью поиска ключевых слов и синонимов, что обеспечивает интерпретируемость и воспроизводимость метода.

In [19]:
import pandas as pd

tracks = pd.read_csv(
    "tracks_with_language_FINAL.csv",
    encoding="utf-8-sig"
)

tracks.columns

/tmp/ipython-input-802214257.py:3: DtypeWarning: Columns (1,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv(


Index(['artist_clean', 'artist_spotify_id', 'name', 'spotify_id',
       'duration_ms', 'explicit', 'popularity', 'album_type', 'album_name',
       'album_spotify_id', 'release_date', 'album_popularity', 'key', 'mode',
       'time_signature', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence',
       'tempo', 'genres', 'artist_list', 'artist_clean_norm', 'language'],
      dtype='object')

In [20]:
import ast
from collections import Counter
import pandas as pd

all_genres = []

for g in tracks["genres"].dropna():
    if not isinstance(g, str):
        continue
    try:
        parsed = ast.literal_eval(g)
        if isinstance(parsed, list):
            all_genres.extend(parsed)
    except:
        pass

len(all_genres)

51743

In [21]:
genre_counts = Counter(all_genres)

genres_df = (
    pd.DataFrame(
        genre_counts.items(),
        columns=["genre", "count"]
    )
    .sort_values("count", ascending=False)
    .reset_index(drop=True)
)

genres_df.head(30)

,genre,count
0,russian pop,1624
1,russian rock,1320
2,russian hip hop,1048
3,classic russian pop,932
4,russian dance pop,720
5,pop,640
6,rap,560
7,classic russian rock,540
8,russian metal,539
9,russian classical piano,512


In [22]:
genres_df.to_csv(
    "all_genres_from_tracks_dataset.csv",
    index=False,
    encoding="utf-8-sig"
)

теперь нормализуем эти жанры

In [23]:
genres_df = pd.read_csv(
    "all_genres_from_tracks_dataset.csv",
    encoding="utf-8-sig"
)

genres_df.head()

,genre,count
0,russian pop,1624
1,russian rock,1320
2,russian hip hop,1048
3,classic russian pop,932
4,russian dance pop,720


In [24]:
def tokenize_genre(g):
    return (
        g.lower()
        .replace("-", " ")
        .replace("_", " ")
        .split()
    )

genres_df["tokens"] = genres_df["genre"].apply(tokenize_genre)

In [25]:
genres_df.to_csv(
    "all_genres_tokenized.csv",
    index=False,
    encoding="utf-8-sig"
)

Для извлечения жанровых предпочтений был сформирован набор из 20 канонических жанров. Словарь жанров был построен на основе анализа жанровых тегов итогового музыкального датасета. Сырые жанры были нормализованы и сопоставлены с каноническими классами, отражающими реальные пользовательские формулировки. Извлечение жанра из текста пользователя осуществляется на основе поиска ключевых токенов, что обеспечивает интерпретируемость и воспроизводимость метода.

In [26]:
genres_df = pd.read_csv(
    "all_genres_tokenized.csv",
    encoding="utf-8-sig"
)

genres_df["tokens"] = genres_df["tokens"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

genres_df.head()

,genre,count,tokens
0,russian pop,1624,"[russian, pop]"
1,russian rock,1320,"[russian, rock]"
2,russian hip hop,1048,"[russian, hip, hop]"
3,classic russian pop,932,"[classic, russian, pop]"
4,russian dance pop,720,"[russian, dance, pop]"


In [27]:
STOP_TOKENS = {
    "russian", "canadian", "swedish", "german", "french",
    "latin", "korean", "japanese", "chinese", "british",
    "classic", "modern", "post", "neo", "new", "old",
}

In [47]:
GENRE_KEYWORDS = {
    # POP / MAINSTREAM
    "pop": {
        "pop", "поп"
    },
    "dance": {
        "dance", "танц"
    },
    "electronic": {
        "electronic", "edm", "house", "techno", "trance", "электрон", "хаус"
    },
    "indie": {
        "indie", "инди"
    },

    # HIP-HOP / URBAN
    "hip_hop": {
        "hip", "hop", "хип", "хоп"
    },
    "rap": {
        "rap", "рэп", "реп"
    },
    "trap": {
        "trap", "трэп", "треп"
    },
    "rnb": {
        "rnb", "r&b", "рнб"
    },

    # ROCK
    "rock": {
        "rock", "рок"
    },
    "metal": {
        "metal", "метал"
    },
    "punk": {
        "punk", "панк"
    },
    "alternative": {
        "alternative", "альтернатив"
    },

    # INSTRUMENTAL / ACADEMIC
    "classical": {
        "classical", "классик", "symphon", "orchestr", "оркест"
    },
    "instrumental": {
        "instrumental", "инструмент", "piano", "пиан"
    },
    "ambient": {
        "ambient", "эмбиент"
    },
    "jazz": {
        "jazz", "джаз"
    },

    # OTHER
    "folk": {
        "folk", "фолк"
    },
    "latin": {
        "latin", "латино"
    },
    "soundtrack": {
        "soundtrack", "саундтрек", "score"
    },
    "blues": {
        "blues", "блюз"
    },
}

In [52]:
import re

def extract_genre_intent(text: str):
    if not isinstance(text, str):
        return []

    text = text.lower()
    tokens = re.findall(r"\w+", text)

    found_genres = set()

    for genre, keywords in GENRE_KEYWORDS.items():
        for token in tokens:
            for kw in keywords:
                if token.startswith(kw):
                    found_genres.add(genre)
                    break

    return list(found_genres)

In [53]:
extract_genre_intent("включи русский рэп")

['rap']

In [54]:
extract_genre_intent("поставь хип хоп")

['hip_hop']

In [55]:
extract_genre_intent("хочу инструментальную музыку")

['instrumental']

In [56]:
extract_genre_intent("включи альтернативную инди музыку")

['alternative', 'indie']

Извлечение жанровых намерений пользователя реализовано на основе правил и словарей ключевых слов. Для обработки морфологических вариаций русского языка используется сопоставление по основе слова (prefix matching), что позволяет корректно учитывать падежные формы без применения морфологического анализа. Словарь жанров включает 20 канонических классов, сформированных на основе анализа жанровых тегов итогового музыкального датасета и отражающих реальные пользовательские формулировки. Такой подход обеспечивает интерпретируемость и воспроизводимость метода.

Мы извлекаем 3 типа намерений:

language — ru / en / instrumental / other

genre — список из 20 канонических жанров

play_intent — есть ли явный запрос на воспроизведение

In [73]:
GENRE_KEYWORDS = {
    # POP / MAINSTREAM
    "pop": {
        "pop", "поп"
    },
    "dance": {
        "dance", "танц"
    },
    "electronic": {
        "electronic", "edm", "house", "techno", "trance", "электрон", "хаус"
    },
    "indie": {
        "indie", "инди"
    },

    # HIP-HOP / URBAN
    "hip_hop": {
        "hip", "hop", "хип", "хоп"
    },
    "rap": {
        "rap", "рэп", "реп"
    },
    "trap": {
        "trap", "трэп", "треп"
    },
    "rnb": {
        "rnb", "r&b", "рнб"
    },

    # ROCK
    "rock": {
        "rock", "рок"
    },
    "metal": {
        "metal", "метал"
    },
    "punk": {
        "punk", "панк"
    },
    "alternative": {
        "alternative", "альтернатив"
    },

    # INSTRUMENTAL / ACADEMIC
    "classical": {
        "classical", "классик", "symphon", "orchestr", "оркест"
    },
    "instrumental": {
        "instrumental", "инструмент", "piano", "пиан"
    },
    "ambient": {
        "ambient", "эмбиент"
    },
    "jazz": {
        "jazz", "джаз"
    },

    # OTHER
    "folk": {
        "folk", "фолк"
    },
    "latin": {
        "latin", "латино"
    },
    "soundtrack": {
        "soundtrack", "саундтрек", "score"
    },
    "blues": {
        "blues", "блюз"
    },
}

In [57]:
LANGUAGE_KEYWORDS = {
    "ru": {"русск", "на русском", "русский"},
    "en": {"английск", "на английском", "english"},
    "instrumental": {"инструмент", "без слов", "без вокала"},
}

In [64]:
PLAY_KEYWORDS = {
    "включи", "поставь", "запусти", "проиграй",
    "хочу", "давай", "воспроизведи",
    "play", "start"
}

In [58]:
def tokenize(text: str):
    return re.findall(r"\w+", text.lower())

In [59]:
def extract_genre_intent(text: str):
    tokens = tokenize(text)
    found = set()

    for genre, keywords in GENRE_KEYWORDS.items():
        for token in tokens:
            for kw in keywords:
                if token.startswith(kw):
                    found.add(genre)
                    break

    return list(found)

In [60]:
def extract_language_intent(text: str):
    tokens = tokenize(text)

    for token in tokens:
        for kw in LANGUAGE_KEYWORDS["instrumental"]:
            if token.startswith(kw):
                return "instrumental"

    for lang in ["ru", "en"]:
        for token in tokens:
            for kw in LANGUAGE_KEYWORDS[lang]:
                if token.startswith(kw):
                    return lang

    return "other"

In [61]:
def extract_play_intent(text: str):
    tokens = tokenize(text)
    return any(token in PLAY_KEYWORDS for token in tokens)

In [62]:
def extract_user_intents(text: str):
    if not isinstance(text, str) or not text.strip():
        return {
            "language": "other",
            "genres": [],
            "play_intent": False
        }

    return {
        "language": extract_language_intent(text),
        "genres": extract_genre_intent(text),
        "play_intent": extract_play_intent(text)
    }

In [65]:
extract_user_intents("включи русский рэп")

{'language': 'ru', 'genres': ['rap'], 'play_intent': True}

In [66]:
extract_user_intents("хочу инструментальную музыку")

{'language': 'instrumental', 'genres': ['instrumental'], 'play_intent': True}

In [67]:
extract_user_intents("электронная музыка")

{'language': 'other', 'genres': ['electronic'], 'play_intent': False}

## Извлечение эмоции пользователя

Эмоциональное состояние пользователя определяется на основе аудиосигнала
до этапа распознавания речи. Для этого используется предварительно обученная
модель распознавания эмоций по аудио, которая анализирует акустические
характеристики сигнала и относит его к одному из фиксированных эмоциональных
классов.

Полученная эмоция далее используется как дополнительный семантический признак
при формировании пользовательского запроса и выборе музыкальных рекомендаций.


In [68]:
EMOTION_CLASSES = {
    "happy",
    "sad",
    "calm",
    "angry",
    "energetic",
    "neutral"
}

In [69]:
def get_emotion_label(emotion):
    """
    Адаптер для эмоции.
    Принимает то, что уже вернула модель,
    и приводит к одному из фиксированных классов.
    """
    if emotion not in EMOTION_CLASSES:
        return "neutral"
    return emotion

In [71]:
LANGUAGE_KEYWORDS = {
    "ru": ["русск", "российск", "по-русск"],
    "instrumental": ["инструментал", "без слов", "фонов", "саундтрек"],
}

In [72]:
PLAY_KEYWORDS = [
    "включи", "поставь", "запусти", "хочу послушать", "проиграй"
]

In [74]:
def extract_user_intent(transcript, emotion_audio):
    return {
        "emotion": emotion_audio,
        "language": extract_language_intent(transcript),
        "genres": extract_genre_intent(transcript),
        "play_intent": extract_play_intent(transcript),
        "transcript": transcript
    }


In [76]:
def extract_language_intent(text):
    for lang, keys in LANGUAGE_KEYWORDS.items():
        if any(k in text for k in keys):
            return lang
    return None

In [77]:
user_intent = extract_user_intent(
    transcript=transcript,
    emotion_audio=emotion_audio
)

user_intent

{'emotion': 'hap',
 'language': None,
 'genres': [],
 'play_intent': False,
 'transcript': 'привет, дела!'}

то есть у нас есть словари намерения

______________________________________________________________

In [83]:
import librosa
import numpy as np
import soundfile as sf
import whisper
from transformers import pipeline

In [84]:
emotion_model = pipeline(
    "audio-classification",
    model="superb/hubert-large-superb-er"
)

whisper_model = whisper.load_model("small")

Device set to use cuda:0


In [85]:
def is_silent(audio, threshold=0.003, min_ratio=0.05):
    rms = librosa.feature.rms(y=audio)[0]
    speech_ratio = (rms > threshold).mean()
    return speech_ratio < min_ratio

def is_too_short(audio, sr, min_duration=1.0):
    return len(audio) / sr < min_duration

def is_noisy(audio, threshold=0.4):
    zcr = librosa.feature.zero_crossing_rate(audio)
    return zcr.mean() > threshold

def validate_audio(audio, sr):
    if is_silent(audio):
        return False, "silence"
    if is_too_short(audio, sr):
        return False, "too short"
    if is_noisy(audio):
        return False, "too noisy"
    return True, "ok"

In [86]:
LANGUAGE_KEYWORDS = {
    "ru": ["русск", "российск", "по-русск"],
    "instrumental": ["инструментал", "без слов", "фонов", "саундтрек"],
}

PLAY_KEYWORDS = [
    "включи", "поставь", "запусти", "проиграй", "хочу послушать"
]

GENRE_SYNONYMS = {
    # POP / MAINSTREAM
    "pop": {
        "pop", "поп"
    },
    "dance": {
        "dance", "танц"
    },
    "electronic": {
        "electronic", "edm", "house", "techno", "trance", "электрон", "хаус"
    },
    "indie": {
        "indie", "инди"
    },

    # HIP-HOP / URBAN
    "hip_hop": {
        "hip", "hop", "хип", "хоп"
    },
    "rap": {
        "rap", "рэп", "реп"
    },
    "trap": {
        "trap", "трэп", "треп"
    },
    "rnb": {
        "rnb", "r&b", "рнб"
    },

    # ROCK
    "rock": {
        "rock", "рок"
    },
    "metal": {
        "metal", "метал"
    },
    "punk": {
        "punk", "панк"
    },
    "alternative": {
        "alternative", "альтернатив"
    },

    # INSTRUMENTAL / ACADEMIC
    "classical": {
        "classical", "классик", "symphon", "orchestr", "оркест"
    },
    "instrumental": {
        "instrumental", "инструмент", "piano", "пиан"
    },
    "ambient": {
        "ambient", "эмбиент"
    },
    "jazz": {
        "jazz", "джаз"
    },

    # OTHER
    "folk": {
        "folk", "фолк"
    },
    "latin": {
        "latin", "латино"
    },
    "soundtrack": {
        "soundtrack", "саундтрек", "score"
    },
    "blues": {
        "blues", "блюз"
    },
}

In [87]:
def extract_language_intent(text):
    for lang, keys in LANGUAGE_KEYWORDS.items():
        if any(k in text for k in keys):
            return lang
    return None

def extract_genre_intent(text):
    found = []
    for genre, keys in GENRE_SYNONYMS.items():
        if any(k in text for k in keys):
            found.append(genre)
    return found

def extract_play_intent(text):
    return any(k in text for k in PLAY_KEYWORDS)

In [88]:
def process_audio_request(audio_path):
    audio, sr = librosa.load(audio_path, sr=16000)

    is_valid, reason = validate_audio(audio, sr)
    if not is_valid:
        return {
            "status": "invalid_audio",
            "reason": reason
        }

    emotion_result = emotion_model(
        {"array": audio, "sampling_rate": sr}
    )
    emotion_audio = emotion_result[0]["label"]
    emotion_confidence = emotion_result[0]["score"]

    rms = np.sqrt(np.mean(audio ** 2))
    if rms > 0:
        audio = audio * (0.1 / rms)

    sf.write("normalized.wav", audio, sr)

    stt_result = whisper_model.transcribe(
        "normalized.wav",
        language="ru",
        task="transcribe",
        condition_on_previous_text=False,
        temperature=0.0
    )

    transcript = stt_result["text"].strip().lower()

    intent = {
        "emotion": emotion_audio,
        "emotion_confidence": round(emotion_confidence, 3),
        "language": extract_language_intent(transcript),
        "genres": extract_genre_intent(transcript),
        "play_intent": extract_play_intent(transcript),
        "transcript": transcript
    }

    return {
        "status": "ok",
        "intent": intent
    }

In [89]:
result = process_audio_request("/content/my_audio_happy.wav")
result

{'status': 'ok',
 'intent': {'emotion': 'hap',
  'emotion_confidence': 0.762,
  'language': None,
  'genres': [],
  'play_intent': False,
  'transcript': 'привет, дела!'}}